In [1]:
import os
from os import listdir
from PIL import Image
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from tensorflow.keras.models import load_model
import numpy as np

import pickle
import cv2

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
MyFaceNet = load_model('facenet_keras.h5')

In [ ]:
folder='data/'
database = {}

for filename in os.listdir(folder):

    path = folder + filename
    gbr1 = cv2.imread(folder + filename)
    
    img = HaarCascade.detectMultiScale(gbr1,1.1,4)
    
    if len(img)>0:
        x1, y1, width, height = img[0]         
    else:
        x1, y1, width, height = 1, 1, 10, 10
        
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)
    
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = MyFaceNet.predict(face)
  
    
    database[os.path.splitext(filename)[0]]=signature

In [ ]:
myfile = open("data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()

In [ ]:
database